<a href="https://colab.research.google.com/github/mehdi-lamrani/llm/blob/main/Semantic_Router_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install langchain_core
!pip install langchain_openai

In [16]:

from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

biology_template = """You are an expert biologist. You excel in explaining biological concepts and processes clearly and thoroughly. \
When confronted with a question outside your expertise, you acknowledge it honestly.

Here is a question:
{query}"""



In [10]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY']= userdata.get('OPENAI_API_KEY')
embeddings = OpenAIEmbeddings()


In [19]:
prompt_templates = [physics_template, math_template, biology_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)


def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]

    template_type = {
        math_template: "Using MATH",
        physics_template: "Using PHYSICS",
        biology_template: "Using BIOLOGY"
    }

    print(template_type.get(most_similar))

    return PromptTemplate.from_template(most_similar)

chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatOpenAI()
    | StrOutputParser()
)

In [22]:
print(chain.invoke("What's a Quasar"))

Using PHYSICS
A black hole is a region in space where gravity is so strong that nothing, not even light, can escape its pull. This happens when a massive star collapses in on itself, creating a singularity with infinite density at its core. The boundary surrounding the singularity is called the event horizon. Black holes can come in different sizes, with supermassive black holes at the centers of galaxies being the largest known examples.


In [27]:
print(chain.invoke("What's a Quasar"))

Using PHYSICS
A quasar is a highly energetic and distant celestial object that emits huge amounts of energy in the form of light and other electromagnetic radiation. Quasars are thought to be powered by supermassive black holes at the centers of galaxies, which accrete large amounts of matter and emit intense radiation as a result. They are some of the brightest and most powerful objects in the universe.


In [25]:
print(chain.invoke("What's Vector Space"))

Using MATH
A vector space is a mathematical structure that consists of a set of elements, called vectors, along with operations of addition and scalar multiplication that satisfy certain properties. These properties include closure under addition and scalar multiplication, associativity and commutativity of addition, distributivity of scalar multiplication over addition, and the existence of an additive identity (the zero vector) and additive inverses for every vector.

In simpler terms, a vector space is a space in which vectors can be added together and multiplied by scalars in a consistent and well-defined way. Vector spaces are fundamental in many areas of mathematics, including linear algebra, functional analysis, and geometry.
